### HPE DACH Introduction to ThreeFold and 3bot technology

Pre-requisites
- install 3bot docker container
- run the jsx install script
- connect to the container
- update the jumpscale libraries
- start kosmos

The jumscale installation documentation is stored [here](https://github.com/threefoldtech/jumpscaleX_core/blob/development/docs/Installation/README.md)

Make sure you have an ssh ID loaded

Recommended is to do the container installation.  Once the container installation completed please start your container by the following command:
```
/tmp/jsx container-shell
```
or
```
ssh root@localhost -p 9000 -A
```
when done
```
kosmos
```
You will then start an interactive python shell with all the jumpscale libraries loaded
```
JSX> 
JSX> exit()
```
Before we start let's check if we have the correct and latest jumpscale libraries

In [ ]:
cd /sandbox/code/github/threefoldtech

for name in `ls`
> do
>     cd $name
>     echo $name
>     git branch
> #   git pull
>     cd ..
> done
home
* master
jumpscaleX_builders
* development
jumpscaleX_core
* development
jumpscaleX_libs
* development
  development_chat_reservation
  master
jumpscaleX_libs_extra
* development
jumpscaleX_threebot
* development
jumpscaleX_weblibs
* development
  master

This should set us up with the latest and greatest.

Set the correct explorere and indentity in your kosmos (3bot brain)

Point to the production explorer
```
j.clients.threebot.explorer_addr_set('explorer.grid.tf')
```
Then check your 3bot indentity:
```
JSX> j.tools.threebot.me.default                                                                                                                                                                                                    
# JSXOBJ:
## jumpscale.threebot.me
ID: 80
 - name                : default
 - tid                 : 92
 - tname               : me
 - email               : weynand@threefold.io
 - pubkey              : 44810e316e49e8d0442b92673627f0f8c2f5f1e23e292c6ba5a294b5aa4057ad
 - admins              : []
```

If the result is empty create or load a threebot identity

Create a 3bot identity use the following simple jumpscale

```
j.tools.threebot.init_my_threebot(
    name='myname.3bot',
    email='myname@gmail.com',
    description="anything can go here")
```

If your have a threebot identity registered with threebot import the the 24 words private seed key

```
j.data.nacl.configure(name='default', privkey_words="<<insert my mnemonic>>", generate=False, reset=True)
```

After completing one of the two options above, testing your threebot identity test your identity by:
```
j.tools.threebot.me.default
```

And to make the mainnet ID mine:
```
me = j.tools.threebot.me.mainnet
```


### Create a network to work with

We need to create a network to work with.  For today we have access to the GEC nodes in Austria and we are going to select a few nodes in the respective farms to create an overlay network on, 

In [ ]:
# load the zero-os sal
zos = j.sal.zosv2

# create an empty reservation data structure (important because the fill method always appends)
r = zos.reservation_create()

execute="yes"



nodes_salzburg = zos.nodes_finder.nodes_search(farm_id=12775) # (IPv6 nodes)
nodes_vienna_1 = zos.nodes_finder.nodes_search(farm_id=82872) # (IPv6 nodes)
nodes_vienna_2 = zos.nodes_finder.nodes_search(farm_id=84041) #(IPv6 nodes)
nodes_belgium = zos.nodes_finder.nodes_search(farm_id=1) # (some IPv4 nodes, to be used as ingress/egress point.  These are not webgatewaysm, just nodes connected to the internet with IPv4 addresses)
nodes_munich = zos.nodes_finder.nodes_search(farm_id=50669) #(IPv6 nodes)

# to show what nodes are available in the 4 farms queried for available nodes:

port_nr=7001

# to show what nodes are available in the 4 farms queried for available nodes:

for i, node in enumerate(nodes_salzburg):
    if zos.nodes_finder.filter_is_up(node):
        iprange = f"172.20.{i+2}.0/24"
        print("Salzburg: zos.network.add_node(network, ",node.node_id,", ",iprange,"wg_port='7001')")
    else:
        print("Node", node.node_id,"is not up")
        
for i, node in enumerate(nodes_vienna_1):
    if zos.nodes_finder.filter_is_up(node):
        iprange = f"172.20.{i+50}.0/24"
        print("Vienna_1: zos.network.add_node(network, ",node.node_id,", ",iprange,"wg_port='7001')")
    else:
        print("Node", node.node_id,"is not up")
        
for i, node in enumerate(nodes_vienna_2):
    if zos.nodes_finder.filter_is_up(node):
        iprange = f"172.20.{i+100}.0/24"
        print("Vienna_2: zos.network.add_node(network, ",node.node_id,", ",iprange,"wg_port='7001')")
    else:
        print("Node", node.node_id,"is not up")

for i, node in enumerate(nodes_munich):
    if zos.nodes_finder.filter_is_up(node):
        iprange = f"172.20.{i+200}.0/24"
        print("Munich: zos.network.add_node(network, ",node.node_id,", ",iprange,"wg_port='7001')")
    else:
        print("Node", node.node_id,"is not up")

for i, node in enumerate(nodes_belgium):
    if zos.nodes_finder.filter_public_ip4(node):
        iprange = f"172.20.{i+150}.0/24"
        print("Belgium: zos.network.add_node(network, ",node.node_id,", ",iprange,"wg_port='7001')")
    else:
        print("Node", node.node_id,"does not have IPv4")


```
Node 7fHSAHEvUGtUcYSqLtpGq8ANssPikTyyHC52FddDYF4Y is up
7fHSAHEvUGtUcYSqLtpGq8ANssPikTyyHC52FddDYF4Y : 172.20.100.0/24
Node FjwyHVvfATkVb4Puh4x6jCMS79TVVgSYagAuZTxWrsbj is up
FjwyHVvfATkVb4Puh4x6jCMS79TVVgSYagAuZTxWrsbj : 172.20.101.0/24
Node 9211BFV7MFwktD2b8jHE9Ub3fHRtaYQyBBfwT9kEKA7q is up
9211BFV7MFwktD2b8jHE9Ub3fHRtaYQyBBfwT9kEKA7q : 172.20.102.0/24
Node CpssVPA4oh455qDxakYhiazgG6t2FT6gAGvmPJMKJL2d is up
CpssVPA4oh455qDxakYhiazgG6t2FT6gAGvmPJMKJL2d : 172.20.103.0/24
Node HkfruwpT1yjx3TTiKn5PVBGFDmnTEqrzz6S36e4rFePb is up
HkfruwpT1yjx3TTiKn5PVBGFDmnTEqrzz6S36e4rFePb : 172.20.104.0/24
Node 9LmpYPBhnrL9VrboNmycJoGfGDjuaMNGsGQKeqrUMSii is up
9LmpYPBhnrL9VrboNmycJoGfGDjuaMNGsGQKeqrUMSii : 172.20.105.0/24
Node HYwvrxCy5z1QPALC5gTAiAMYWLH7orQ8xWEDbEyLiJV7 is up
HYwvrxCy5z1QPALC5gTAiAMYWLH7orQ8xWEDbEyLiJV7 : 172.20.106.0/24
Node CBDY1Fu4CuxGpdU3zLL9QT5DGaRkxjpuJmzV6V5CBWg4 is up
CBDY1Fu4CuxGpdU3zLL9QT5DGaRkxjpuJmzV6V5CBWg4 : 172.20.107.0/24
Node Hb6oVe2B5v9UBzDcDeQfZGn5bwFeM2R3rJh6U93AWfiN is up
Hb6oVe2B5v9UBzDcDeQfZGn5bwFeM2R3rJh6U93AWfiN : 172.20.108.0/24
```

Here we need to design the network we would like to use.  For the purpose of today it might be best to inclulde all vienna_1 and all salzburg nodes plus one IPv4 nodes in Belgium

In [ ]:
nodes_salzburg = zos.nodes_finder.nodes_search(farm_id=12775) # (IPv6 nodes)
nodes_vienna_1 = zos.nodes_finder.nodes_search(farm_id=82872) # (IPv6 nodes)
nodes_vienna_2 = zos.nodes_finder.nodes_search(farm_id=84041) #(IPv6 nodes)
nodes_belgium = zos.nodes_finder.nodes_search(farm_id=1) # (some IPv4 nodes, to be used as ingress/egress point.  These are not webgatewaysm, just nodes connected to the internet with IPv4 addresses)
nodes_munich = zos.nodes_finder.nodes_search(farm_id=50669) #(IPv6 nodes)

# nodes_all = nodes_salzburg + nodes_vienna_1 + nodes_belgium
nodes_all = nodes_salzburg[:3] + nodes_vienna_1[:3] + nodes_belgium[:3]

network = zos.network.create(r, ip_range="172.20.0.0/16", network_name="hpe_weynand_testnet_20")

# make sure to set a new port
for i, node in enumerate(nodes_all):
    if zos.nodes_finder.filter_is_up(node):
        iprange = f"172.20.{i+100}.0/24"
        print("Node", node.node_id,"is up")
        zos.network.add_node(network, node.node_id , iprange, wg_port='8020')
        print(node.node_id, ":", iprange)
    else:
        print("Node", node.node_id,"is not up")

In [ ]:
# Enter here the node_id for the node that is the IPv4 bridge to create the wireguard config.

wg_config = zos.network.add_access(network, 'CBDY1Fu4CuxGpdU3zLL9QT5DGaRkxjpuJmzV6V5CBWg4', '172.20.100.0/24', ipv4=True)

print("wireguard configuration")
print(wg_config)
print("provisioning result")
print(result)

In [ ]:
# Make a reservation for the network for a 24 hours.
identity = j.tools.threebot.me.mainnet

import time

expiration = j.data.time.epoch + (24*60*60)
# register the reservation
rid = zos.reservation_register(r, expiration, identity=me)
time.sleep(5)
# inspect the result of the reservation provisioning
result = zos.reservation_result(rid)

# test your network deployment
for i in 100 101 102 103 104 105 106 107 108 110 201 202 204 206 207 208 209 210 155; do ping -c 1 172.20.$i.1; done

In [ ]:
# Copy the result and create a wireguard tunnel on your device.
# Save the file to wgHPE.conf

sudo -i
wg-quick up ./wgHPE.conf

### Networking done - now we can deploy containers.



In [ ]:
# Add data to method to what to deploy.  Example is code server
# Before deploying - adapt node_id and node ip-address, Interactive True or False
r = zos.reservation_create()
zos.container.create(reservation=r,
                    node_id='2fi9ZZiBGW4G9pnrN656bMfW6x55RSoHDeMrd9pgSA8T', # one of the node_id that is part of the network
                    network_name='ambassador_testnet_0', # this assume this network is already provisioned on the node
                    ip_address='172.20.209.20', # part of ip_range you reserved for your network xxx.xxx.1.10
                    flist='https://hub.grid.tf/weynandkuijpers.3bot/codercom-code-server-latest.flist', # flist of the container you want to install
                    interactive=False,         # True only if corex_connect required, default false
                  # env={},                   # field for parameters like config 
                    entrypoint='/usr/local/bin/code-server')

# expiration = j.data.time.epoch + (3600 * 24 * 365)
# reserve until now + (x) seconds
expiration = j.data.time.epoch + (10*60)
# register the reservation
rid = zos.reservation_register(r, expiration, identity=me)
time.sleep(5)
# inspect the result of the reservation provisioning
result = zos.reservation_result(rid)

# to check the reservation in the blockchain database
zos.reservation_get(rid)
zos.reservation_live(rid)
zos.reservation_cancel(rid)


In [ ]:
http://172.20.201.16:7681/
http://172.20.201.16:7681/api/process/start?arg[]=/bin/sh

### S3 deployment

```
Node 7fHSAHEvUGtUcYSqLtpGq8ANssPikTyyHC52FddDYF4Y is up
7fHSAHEvUGtUcYSqLtpGq8ANssPikTyyHC52FddDYF4Y : 172.20.100.0/24
Node FjwyHVvfATkVb4Puh4x6jCMS79TVVgSYagAuZTxWrsbj is up
FjwyHVvfATkVb4Puh4x6jCMS79TVVgSYagAuZTxWrsbj : 172.20.101.0/24
Node 9211BFV7MFwktD2b8jHE9Ub3fHRtaYQyBBfwT9kEKA7q is up
9211BFV7MFwktD2b8jHE9Ub3fHRtaYQyBBfwT9kEKA7q : 172.20.102.0/24
Node CpssVPA4oh455qDxakYhiazgG6t2FT6gAGvmPJMKJL2d is up
CpssVPA4oh455qDxakYhiazgG6t2FT6gAGvmPJMKJL2d : 172.20.103.0/24
Node HkfruwpT1yjx3TTiKn5PVBGFDmnTEqrzz6S36e4rFePb is up
HkfruwpT1yjx3TTiKn5PVBGFDmnTEqrzz6S36e4rFePb : 172.20.104.0/24
Node 9LmpYPBhnrL9VrboNmycJoGfGDjuaMNGsGQKeqrUMSii is up
9LmpYPBhnrL9VrboNmycJoGfGDjuaMNGsGQKeqrUMSii : 172.20.105.0/24
Node HYwvrxCy5z1QPALC5gTAiAMYWLH7orQ8xWEDbEyLiJV7 is up
HYwvrxCy5z1QPALC5gTAiAMYWLH7orQ8xWEDbEyLiJV7 : 172.20.106.0/24
Node CBDY1Fu4CuxGpdU3zLL9QT5DGaRkxjpuJmzV6V5CBWg4 is up
CBDY1Fu4CuxGpdU3zLL9QT5DGaRkxjpuJmzV6V5CBWg4 : 172.20.107.0/24
Node Hb6oVe2B5v9UBzDcDeQfZGn5bwFeM2R3rJh6U93AWfiN is up
Hb6oVe2B5v9UBzDcDeQfZGn5bwFeM2R3rJh6U93AWfiN : 172.20.108.0/24
```

In [ ]:
# ----------------------------------------------------------------------------------
# Point to the testnet explorere and load the ZOS system abstraction layer (SAL)
# ----------------------------------------------------------------------------------
j.clients.threebot.explorer_addr_set('explorer.testnet.grid.tf')

# load the zero-os sal
zos = j.sal.zosv2

# ----------------------------------------------------------------------------------
# Find nodes in a Farm to deploy ZDB's on.
# ----------------------------------------------------------------------------------
r = zos.reservation_create()

# first find the node where to reserve 0-db namespaces.  Select all the salzburg nodes
nodes_salzburg = zos.nodes_finder.nodes_search(farm_id=12775) # (IPv6 nodes)
nodes_vienna_1 = zos.nodes_finder.nodes_search(farm_id=82872) # (IPv6 nodes)

nodes_all = nodes_salzburg[1:3] + nodes_vienna_1[:3]

# nodes = zos.nodes_finder.nodes_search(sru=10)
#nodes = list(filter(zos.nodes_finder.filter_is_up,nodes))
#nodes = nodes[:2]
        
for i, node in enumerate(nodes_all):
    if zos.nodes_finder.filter_is_up(node):
        print(i, ":", node.node_id)
j.
# ----------------------------------------------------------------------------------
# create a reservation for ZDB's - the low level disk interfaces
# ----------------------------------------------------------------------------------
r = zos.reservation_create()
# reservation some 0-db namespaces
for node in nodes_all:
    zos.zdb.create(
        reservation=r,
        node_id=node.node_id,
        size=10,
        mode='seq',
        password='supersecret',
        disk_type="SSD",
        public=False)

# ----------------------------------------------------------------------------------
# Write reservation for ZDB's in BCDB - the low level disk interfaces
# ----------------------------------------------------------------------------------   
expiration = j.data.time.epoch + (10*60)
# register the reservation
rid = zos.reservation_register(r, expiration, identity=me)
time.sleep(5)

results = zos.reservation_result(rid)
while len(results) < len(nodes_all):
    time.sleep(2)
    results = zos.reservation_result(rid)

# ----------------------------------------------------------------------------------
# read the IP address of the 0-db namespaces after they are deployed
# we will need these IPs when creating the minio container
# As we are inside the 3bot - we can pass the access passwords in plain text.
# ----------------------------------------------------------------------------------
namespace_config = []
for result in results:
    cfg = f"{result.data_json['Namespace']}:supersecret@[{result.data_json['IP']}]:{result.data_json['Port']}"
    namespace_config.append(cfg)

    
# ----------------------------------------------------------------------------------
# With the low level disk managers done and the IP adresses discovered we can now build
# the reservation for the min.io S3 interface.
# ----------------------------------------------------------------------------------
r = zos.reservation_create()

# Make sure to adjust the node_id and network name to the appropriate in copy / paste mode :-)
zos.container.create(reservation=r,
    node_id="A7FmQZ72h7FzjkJMGXmzLDFyfyxzitDZYuernGG97nv7",
    network_name='ambassador_testnet_0',
    ip_address='172.20.201.16',
    flist='https://hub.grid.tf/azmy.3bot/minio.flist',
    interactive=False, 
    entrypoint='/bin/entrypoint',
    cpu=2,
    memory=2048,
    env={
        "SHARDS":','.join(namespace_config),
        "DATA":"3",
        "PARITY":"2",
        "ACCESS_KEY":"minio",
        "SECRET_KEY":"passwordpassword",
        })

# ----------------------------------------------------------------------------------
# Write reservation for min.io container in BCDB - end user interface
# ----------------------------------------------------------------------------------      
rid = zos.reservation_register(r, j.data.time.epoch+(10*60))
rid = zos.reservation_register(r, expiration, identity=me)
# ----------------------------------------------------------------------------------

### Deploy K3S


In [ ]:
# Have a list of available nodes in the network available
        
zos = j.sal.zosv2
r = zos.reservation_create()

cluster_secret = 'supersecret'
size = 1

# Make sure to put the network name created.

network_name = 'zaibon_testnet_0'

# Enter your public ssh key ~$HOME/.ssh/id_rsa.pub
# sshkeys = ['ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIMtml/KgilrDqSeFDBRLImhoAfIqikR2N9XH3pVbb7ex zaibon@tesla']
sshkeys = [``]

# Make sure to select the correct nodes + node_ip
master = zos.kubernetes.add_master(
    reservation=r,
    node_id='2fi9ZZiBGW4G9pnrN656bMfW6x55RSoHDeMrd9pgSA8T',
    network_name=network_name,
    cluster_secret=cluster_secret,
    ip_address='172.24.1.20',
    size=size,
    ssh_keys=sshkeys)

#repeat with different nodes and IP's for multiple workers.

worker = zos.kubernetes.add_worker(
    reservation=r,
    node_id='72CP8QPhMSpF7MbSvNR1TYZFbTnbRiuyvq5xwcoRNAib',
    network_name=network_name,
    cluster_secret=cluster_secret,
    ip_address='172.24.2.20',
    size=size,
    master_ip=master.ipaddress,
    ssh_keys=sshkeys)

expiration = j.data.time.epoch + (3600 * 24 * 365)
# register the reservation
rid = zos.reservation_register(r, expiration, identity=me)
time.sleep(120)
# inspect the result of the reservation provisioning
result = zos.reservation_result(rid)

print("provisioning result")
print(result)